<a href="https://colab.research.google.com/github/javohir-ergashboyev/NLP/blob/main/Stop_words_and_n_gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy

In [2]:
nlp=spacy.load('en_core_web_sm')
text=('We do not have to speak in russian as long as we are citizen of Uzbekistan living in homeland')
doc=nlp(text)
for token in doc:
  if token.is_stop:
    print(token)

We
do
not
have
to
in
as
as
we
are
of
in


In [3]:
from spacy.lang.en import STOP_WORDS
len(STOP_WORDS)

326

In [4]:
def preprocess(text):
  doc=nlp(text)
  non_stop_words=[token.text for token in doc if not token.is_stop and not token.is_punct]
  return ' '.join(non_stop_words)

In [5]:
preprocess('I should have believe when you said you were victim there')

'believe said victim'

In [6]:
import pandas as pd

In [8]:
df=pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
df['New Message']=df['Message'].apply(preprocess)

In [10]:
df.head()

,Category,Message,New Message
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy Available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,U dun early hor U c
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah think goes usf lives


In [11]:
df[['Message', 'New Message']].head(1)

,Message,New Message
0,"Go until jurong point, crazy.. Available only ...",jurong point crazy Available bugis n great wor...


# n-grams

countvectorizer

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
text='Hello, my name is Luis'
v=CountVectorizer(ngram_range=(1,3))
v.fit([text])
v.vocabulary_

{'hello': 0,
 'my': 6,
 'name': 9,
 'is': 3,
 'luis': 5,
 'hello my': 1,
 'my name': 7,
 'name is': 10,
 'is luis': 4,
 'hello my name': 2,
 'my name is': 8,
 'name is luis': 11}

In [12]:
def preprocess2(text):
  doc=nlp(text)
  filtered_text=[i.lemma_ for i in doc if not i.is_stop and not i.is_punct]
  return ' '.join(filtered_text)

In [13]:
preprocess2('Hello, my name is Luis')

'hello Luis'

In [21]:
list_str=[
    'Ronaldo came over all the way from London',
    'None can insult your way of looking at world',
    'Modric came over all the way from London'
]

In [22]:
data=[preprocess2(i) for i in list_str]
data

['Ronaldo come way London', 'insult way look world', 'Modric come way London']

In [23]:
v=CountVectorizer(ngram_range=(1,2))
v.fit(data)
v.vocabulary_

{'ronaldo': 9,
 'come': 0,
 'way': 11,
 'london': 4,
 'ronaldo come': 10,
 'come way': 1,
 'way london': 12,
 'insult': 2,
 'look': 5,
 'world': 14,
 'insult way': 3,
 'way look': 13,
 'look world': 6,
 'modric': 7,
 'modric come': 8}

In [25]:
v.transform(['Ronaldo come London']).toarray()

array([[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]])

In [27]:
v.transform(['Kroos come London']).toarray()

array([[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Building a model using our spam.csv file

In [30]:
df=pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [31]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [34]:
min_num=747
df_ham=df[df['Category']=='ham'].sample(min_num, random_state=20)
df_ham.head()

,Category,Message
1600,ham,"Yeah probably, I still gotta check out with leo"
2026,ham,"Yes obviously, but you are the eggs-pert and t..."
1148,ham,Ok... Help me ask if she's working tmr a not?
5296,ham,Are u coming to the funeral home
57,ham,"Sorry, I'll call later in meeting."


In [36]:
df_spam=df[df['Category']=='spam']
df_spam.head()

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [37]:
df_new=pd.concat([df_ham,df_spam], axis=0)

In [39]:
df_new.sample(5)

,Category,Message
3104,ham,"U so lousy, run already come back then half de..."
797,spam,"Orange customer, you may now claim your FREE C..."
2751,ham,Ok but tell me half an hr b4 u come i need 2 p...
2642,spam,"You are guaranteed the latest Nokia Phone, a 4..."
5109,ham,Oh you got many responsibilities.


In [41]:
df_new['spam']=df['Category'].map({'spam':1, 'ham':0})
df_new.sample(5)

,Category,Message,spam
5269,spam,"If you don't, your prize will go to another cu...",1
5027,spam,Ur cash-balance is currently 500 pounds - to m...,1
2739,ham,I sent you the prices and do you mean the &lt...,0
1196,spam,You have 1 new voicemail. Please call 08719181503,1
1423,spam,Congratulations ur awarded either £500 of CD g...,1


In [42]:
from sklearn.model_selection import train_test_split
xtrain, xtest,ytrain,ytest=train_test_split(df_new['Message'], df_new['spam'], test_size=0.2,
                                            random_state=5, stratify=df_new['spam'])

In [43]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf=Pipeline([
    ('vectorizer', CountVectorizer()),
    ('naive', MultinomialNB())
])
clf.fit(xtrain,ytrain)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('naive', MultinomialNB())])

In [44]:
ypred=clf.predict(xtest)

In [45]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       149
           1       0.98      0.97      0.97       150

    accuracy                           0.97       299
   macro avg       0.97      0.97      0.97       299
weighted avg       0.97      0.97      0.97       299



In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf=Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('naive', MultinomialNB())
])
clf.fit(xtrain,ytrain)
ypred=clf.predict(xtest)
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97       149
           1       0.97      0.97      0.97       150

    accuracy                           0.97       299
   macro avg       0.97      0.97      0.97       299
weighted avg       0.97      0.97      0.97       299

